In [2]:
import pandas
import implicit

import scipy as sp
import numpy as np
import turicreate as tc

In [3]:
df = pandas.read_csv("user_item_test.csv")
df

,user_id,item_id,user,item,rating,rating2
0,0,0,amit,seahawks,3,1
1,0,1,amit,sf giants,3,1
2,0,2,amit,warriors,3,1
3,1,3,aryana,patriots,3,1
4,1,2,aryana,warriors,5,1
5,2,4,mital,steelers,5,1
6,2,5,mital,penguins,4,1
7,2,6,mital,pirates,3,1
8,2,7,mital,celtics,1,1
9,3,4,kaiden,steelers,5,1


In [20]:
# just the matrix
df_pivot = df.pivot_table(values='rating', index='item', columns='user', aggfunc=len, fill_value=0)
item_user_data = df_pivot.values

sdata = sp.sparse.csr_matrix(item_user_data)
print (sdata)


  (0, 4)	1
  (1, 2)	1
  (2, 4)	1
  (3, 4)	1
  (4, 0)	1
  (5, 0)	1
  (6, 3)	1
  (6, 4)	1
  (7, 0)	1
  (7, 1)	1
  (7, 2)	1


In [5]:
df.pivot_table(values='rating', index='item', columns='user', aggfunc=len, fill_value=0)

user,amit,anika,aryana,kaiden,mital
item,,,,,
celtics,0,0,0,0,1
patriots,0,0,1,0,0
penguins,0,0,0,0,1
pirates,0,0,0,0,1
seahawks,1,0,0,0,0
sf giants,1,0,0,0,0
steelers,0,0,0,1,1
warriors,1,1,1,0,0


## try implicit package


In [21]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(sdata)


In [22]:
def recs(recommendations, df, userid):
    user = df.columns[userid]
    
    print("recomemndations for %d: %s" % ( userid, user)) 
    for rr in recommendations:
        item = df.index[rr[0]]
        score = rr[1]
        print (item, score )
    

In [25]:
userid = 1


# exclude what you already like
col = df_pivot.columns[userid]
print(userid, df_pivot.columns[userid])
user_filter  = (df_pivot[col] > 0)
prev_items = df_pivot[user_filter].index
df_items = pandas.DataFrame(df_pivot.index)

item_filter = df_items.isin(prev_items)
exclude_items = df_items[item_filter].dropna().index.values
list(exclude_items)

1 anika


[7]

In [26]:

# recommend items for a user
user_items = sdata.T

recommendations = model.recommend(userid, user_items, filter_items=list(exclude_items))
print(recommendations)
recs(recommendations, df_pivot, userid)

[(1, 0.0039256737), (4, 0.0027464628), (5, 0.0027464628), (6, 0.00011751801), (3, 5.4366887e-05), (2, 5.4344535e-05)]
recomemndations for 1: anika
patriots 0.0039256737
seahawks 0.0027464628
sf giants 0.0027464628
steelers 0.00011751801
pirates 5.4366887e-05
penguins 5.4344535e-05


In [10]:
recommendations

[(2, 0.0016410723),
 (3, 0.0016401969),
 (1, 0.00019751489),
 (4, 2.7555972e-05),
 (5, 2.7529895e-05),
 (7, -3.0100346e-06)]

In [11]:
itemid = recommendations[0][0]
?model.explain(userid, user_items, itemid)

Object `model.explain(userid, user_items, itemid)` not found.


## try turi/graphlab package

https://turi.com/

In [27]:
sf = tc.SFrame(df)
sf

user_id,item_id,user,item,rating,rating2
0,0,amit,seahawks,3,1
0,1,amit,sf giants,3,1
0,2,amit,warriors,3,1
1,3,aryana,patriots,3,1
1,2,aryana,warriors,5,1
2,4,mital,steelers,5,1
2,5,mital,penguins,4,1
2,6,mital,pirates,3,1
2,7,mital,celtics,1,1
3,4,kaiden,steelers,5,1


In [13]:
?tc.recommender.create

In [28]:
sf = tc.SFrame(df)
model = tc.recommender.create(sf, 'user_id', 'item_id') #, "rating")

Warning: Ignoring columns user, item, rating, rating2;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 11 observations with 5 users and 8 items.

...

| 3.623ms                             | 100              | 8               |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.00598s

In [29]:
model.recommend([1])

user_id,item_id,score,rank
1,0,0.1666666567325592,1
1,1,0.1666666567325592,2
1,4,0.0,3
1,5,0.0,4
1,6,0.0,5
1,7,0.0,6


In [30]:
model.recommend([3]).join(sf, on=['item_id']).sort('score')

user_id,item_id,score,rank,user_id.1,user,item,rating,rating2
3,0,0.0,4,0,amit,seahawks,3,1
3,1,0.0,6,0,amit,sf giants,3,1
3,2,0.0,5,0,amit,warriors,3,1
3,3,0.0,7,1,aryana,patriots,3,1
3,2,0.0,5,1,aryana,warriors,5,1
3,2,0.0,5,4,anika,warriors,5,1
3,6,0.5,2,2,mital,pirates,3,1
3,7,0.5,1,2,mital,celtics,1,1
3,5,0.5,3,2,mital,penguins,4,1


## turi ranking factorization

In [31]:
model = tc.ranking_factorization_recommender.create(sf, 'user_id', 'item_id')

Preparing data set.

Data has 11 observations with 5 users and 8 items.

Data prepared in: 0.032393s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.138929

Final training Predictive Error: 0.0190039

In [40]:
?model.recommend([1])

In [33]:
model.recommend([1]).join(sf, on=['item_id']).sort('score')

user_id,item_id,score,rank,user_id.1,user,item,rating,rating2
1,1,0.12696840475751295,6,0,amit,sf giants,3,1
1,0,0.13060150292614756,5,0,amit,seahawks,3,1
1,5,0.23782044518441509,4,2,mital,penguins,4,1
1,6,0.24151852235768734,3,2,mital,pirates,3,1
1,7,0.2648532517804524,2,2,mital,celtics,1,1
1,4,0.3315815610012338,1,2,mital,steelers,5,1
1,4,0.3315815610012338,1,3,kaiden,steelers,5,1


In [19]:
sf

user_id,item_id,user,item,rating,rating2
0,0,amit,seahawks,3,1
0,1,amit,sf giants,3,1
0,2,amit,warriors,3,1
1,3,aryana,patriots,3,1
1,2,aryana,warriors,5,1
2,4,mital,steelers,5,1
2,5,mital,penguins,4,1
2,6,mital,pirates,3,1
2,7,mital,celtics,1,1
3,4,kaiden,steelers,5,1
